<a href="https://colab.research.google.com/github/nagaokayama/captioning_scripts/blob/main/Text_cordinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 0. google driveのファイルを使用する場合はまずこのスクリプトを実行してください {display-mode: "form"}
# @markdown ←●に▶が入ってるマークが「セルを実行ボタン」です。これを押して実行\
# @markdown グーグルドライブの中身はファイルブラウザのgdrive/MyDriveに共有されておりここから閲覧できます


# @markdown 左端にあるフォルダのマークをクリックしてファイルブラウザを開き\
# @markdown 目的のフォルダにマウスオーバーすると表示される縦に3点が並んだマークをクリック→「パスをコピー」でファイルパスをコピーできます
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:

# @title 0.必要なモジュールのインポートと関数の設定　{display-mode: "form"}
# @markdown ステップ1: セルを実行\
# @markdown 作業をする前にこのセルを実行してください


import os
import csv
import re
import glob
import chardet

from PIL import Image


def extract_fileName(imagesFolder):
    # 引数にいれたフォルダの中から画像ファイルを探してファイルのリストを返す
    # globで簡単に全部のファイルをリストした後にre.searchで画像ファイルだけを抽出する

    fileNameList = []

    tmpFiles = glob.glob(f"{imagesFolder}/*.*")
    tmpFiles.sort()

    if len(tmpFiles) == 0:
        return -1

    for name in tmpFiles:
        # 画像ファイル以外の拡張子を持つファイルを除外
        pattern = re.compile(".*\.(png|jpg|jpeg|gif|webp|tiff)$",re.IGNORECASE)
        basename = os.path.basename(name)

        ismatch = pattern.search(basename)
        if ismatch:
            tmp = ismatch.group(0).split(".")[0] # 拡張子を削除
            fileNameList.append(tmp)

    if len(fileNameList) == 0:
        return -1

    return fileNameList

def detect_encoding(file_path):
    # エンコードの自動判定 100％の精度ではないらしい

    with open(file_path, 'rb') as file:
        rawdata = file.read()
    result = chardet.detect(rawdata)
    return result['encoding']


In [ ]:

# 共通キャプションを持ったcsvファイルを生成
# UIはたいぷスクリプトで書いてるみたい

# @title 画像名と共通キャプションを各行に書き込んだcsvを生成　{display-mode: "form"}
# @markdown ステップ1:画像ファイルのフォルダを指定
image_folder_path = ''  # @param {type:"string"}

# @markdown ステップ2: すべての画像に書き込むキャプションをカンマor読点区切りで入力\
# @markdown 共通キャプションがない場合は空欄にしてください
# unicodeでそのまま書かれちゃうけどrを入れてなければちゃんと解釈されるっぽい？
common_caption = '' # @param {type:"string"}

# @markdown ステップ3: CSVファイルの書き出し先を指定
# CSVファイルにデータを書き込み
csv_file_path = ''  # @param {type:"string"}

# @markdown microsoft エクセルでCSVファイルを読み込む場合はチェックをつける（shift-jisでcsvが生成されます）
export_with_shiftjis = False # @param {type:"boolean"}

# @markdown 必要事項が書けたらセルを実行

def generate_csv_caption(image_files,common_captions="", encoding = "utf8" ):
    common_caption_words = re.split(r',|、', common_captions)
    data = []

    for image_file in image_files:
        # 画像名と共通キャプションをまとめてcsv用の配列を生成
        data.append([image_file] + common_caption_words)

    data.sort()

    csv_fullpath = os.path.join(csv_file_path, 'imageAndCaptions_data.csv')
    with open(csv_fullpath, 'w', newline='', encoding = encoding) as csv_file:
        csv_writer = csv.writer(csv_file)

        # データを書き込む
        csv_writer.writerows(data)

    print(f"CSVファイルが {csv_fullpath} に生成されました。")


# HACK:トグルでエンコードを切り替えたい。こいつはあんまりきれいじゃないと思う
if export_with_shiftjis :
    encode_mode = "shift-jis"
else:
    encode_mode = "utf8"


imageList = extract_fileName(image_folder_path)

if imageList == -1:
    print(f"エラー: {image_folder_path} に画像ファイルが存在しません")
else:
    generate_csv_caption(imageList,common_caption,encode_mode)



In [ ]:

# @title csvファイルから各画像用キャプションテキストファイルを生成　{display-mode: "form"}
# @markdown ステップ1:  csvファイルを指定
csv_file_path = '/content/imageAndCaptions_data.csv' # @param {type:"string"}

# markdown ステップ1.オプション:  画像フォルダを指定
images_folder_path = '/content/imageAndCaptions_data.csv' # param {type:"string"}

# csv_file_path = '/content/Untitled Folder/image_data.csv' # param type{"string"}
# @markdown ステップ2: キャプションファイルの書き出しフォルダを指定
text_folder_path = '/content'  # @param {type:"string"}

# @markdown 「セルを実行」ボタンを押してキャプションテキストファイルを生成


def getFilenamesAndTagsFromCsv(csv_file_path=csv_file_path):

    encoding = detect_encoding(csv_file_path)
    print(f"The detected encoding is: {encoding}")

    with open(csv_file_path, 'r', newline='', encoding=encoding) as csv_file:
        csv_reader = csv.reader(csv_file)
        captionsList = []
        imagenameList = []
        # rowは縦だっけ？
        for row in csv_reader:
            # 各行の要素について処理
            imagenameList.append(row[0])
            # row[0]以外のすべての要素をタグとして配列に入れる
            # 空の要素をテキストファイルに入れないように排除
            tmp = [i for i in row[1:] if i != ""]
            captionsList.append(tmp)

    return imagenameList, captionsList


def generateTextFileFromList(imagenameList, captionsList):
    # リストを百要素ごとに分割する
    # list = list[siki for i in is]
    for n, imagename in enumerate(imagenameList):
        # captionsのためにわざわざenumerate()つけるのなんかやだかも
        captions = captionsList[n]
        text_file_path = f'{text_folder_path}/{imagename}.txt'

        concat_captions = "、".join(captions)
        with open(text_file_path, 'w') as text_file:
            text_file.write(concat_captions)

    print(f"キャプションファイルを {text_folder_path}に保存済み")

imagenameList, captionsList = getFilenamesAndTagsFromCsv(csv_file_path)
generateTextFileFromList(imagenameList, captionsList)

In [ ]:

# @title 各画像ファイル名をテキストファイルに書き込む {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

# @markdown 左端にあるフォルダのマークをクリックしてファイルブラウザを開き\
# @markdown  目的のフォルダにマウスオーバーすると表示される縦に3点が並んだマークをクリック→「パスをコピー」でファイルパスをコピーできます

# @markdown 画像の入っているフォルダを指定
image_folder = "" #@param {type:"string"}
# @markdown 「セルを実行」ボタンを押してテキストファイルを生成\
# @markdown テキストファイルはimage_folderに指定したフォルダに作成されます。


#フォルダとテキストのファイル名は自動で決める

def export_ImageFileName(textstring):
  text = ""
  for i in textstring:
    # 配列の各要素を指定した文字列を間に挟みつつ全結合
    text = "\n".join(textstring)

  # print(text)
  filepath = os.path.join(image_folder,"imageFileName_List.txt")
  with open(filepath, mode = "w") as f:
    f.write(text)
  return f"テキストファイルを{filepath}に生成"

# 関数を実行
tmp = extract_fileName(image_folder)
export_ImageFileName(tmp)

In [ ]:
# @title 開発中　{display-mode: "form"}

# @markdown 画像のフォルダ
dir_name = '/content/drive/MyDrive/My_Finetuing/robo' # @param {type:"string"}
# @markdown リサイズ後の画像の保存先を指定
new_dir_name = '/content/drive/MyDrive/My_Finetuing/resizeRobo' # @param {type:"string"}

files = os.listdir(dir_name)

for file in files:
    img = Image.open(os.path.join(dir_name, file))
    if (1024 * img.height // img.width)<512:
        img_resize = img.resize((1024, 1024 * img.height // img.width)) #長辺が1024
    else :
        img_resize = img.resize((1024, 1024 * img.height // img.width)) #かきかけ
        #短辺が512

        img_resize.save(os.path.join(new_dir_name, file))